# Reproduction of Geography 120 Gerrymandering and Area-Weighted Reaggregation Lab

### Authors

Grace Sokolow\*, gsokolow@middlebury.edu, @gsokolow, Middlebury College (Undergraduate)

\* Corresponding author and creator


Version 0.1.0 \| First Created November 30, 2023 \| Updated November 30, 2023



### NOTE

can delete any extraneous bits and then bring in all the analysis plan info via code or copy and paste; see the original template format for what's suggested

use markdown (here()) -> ask Liam

## Abstract


## Study design

## Computational environment

This study uses Python 3.11.5. Dependencies are linked in procedure/environment/requirements.txt


In [1]:
# report python version and install required packages
# switch if statement from True to False once packages have been installed
if True:
    !python -V
    !pip install -r ../environment/requirements.txt

Python 3.11.5
Defaulting to user installation because normal site-packages is not writeable


In [2]:
# Now, let's import what we need
import geopandas as gpd
import pandas as pd

In [3]:
# check version
pip.freeze

NameError: name 'pip' is not defined

### Read in Data Sources

Let's open each data source and examine its variables.

In [ ]:
cd_2016 = gpd.read_file("../../data/raw/public/givens/2016_Contingent_Congressional_Plan_Corrected.shp")
cd_2019 = gpd.read_file("../../data/raw/public/givens/C-Goodwin-A-1-TC.shp")
pc2016 = gpd.read_file("../../data/raw/public/givens/precincts.shp")

print(pc2016.head(0))

In [ ]:
pc2016.dtypes

In [ ]:
pc2016["rep"].min()

### Analysis

Describe the methods of analysis that will directly test the hypotheses or provide results to answer the research questions.
This section should explicitly define any spatial / statistical *models* and their *parameters*, including *grouping* criteria, *weighting* criteria, and *significance thresholds*.
Also explain any follow-up analyses or validations.



## Results

Describe how results are to be presented.



## Discussion

Describe how the results are to be interpreted *vis a vis* each hypothesis or research question.



## Integrity Statement

Include an integrity statement - The authors of this preregistration state that they completed this preregistration to the best of their knowledge and that no other preregistration exists pertaining to the same hypotheses and research.
If a prior registration *does* exist, explain the rationale for revising the registration here.



# Acknowledgements

- `Funding Name`: name of funding for the project
- `Funding Title`: title of project grant
- `Award info URI`: web address for award information
- `Award number`: award number

This report is based upon the template for Reproducible and Replicable Research in Human-Environment and Geographical Sciences, DOI:[10.17605/OSF.IO/W29MQ](https://doi.org/10.17605/OSF.IO/W29MQ)

## References